In [40]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from utils import MyTabNetClassifier
from acv_explainers.utils import *
from ares import AReS
from cet import CounterfactualExplanationTree
from ce import ActionExtractor
from utils import DatasetHelper, submodular_picking, DATASETS_NAME
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [41]:
# Load Diabetes datasets

seed = 2022
np.random.seed(0)

X = pd.read_csv('data/diabetes/train.csv')
y = X.Outcome

X.drop(['Id', 'Outcome'], axis=1, inplace=True)
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.20, random_state=seed)

In [42]:
X.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
dtype: object

In [43]:
# Train IsolationForest for outlier score

from sklearn.ensemble import IsolationForest

isolation = IsolationForest()
isolation.fit(X_tr)

IsolationForest()

In [44]:
# Train the model to explain

mdl = LGBMClassifier(n_estimators=50, num_leaves=8)
mdl.fit(X_tr, y_tr)

LGBMClassifier(n_estimators=50, num_leaves=8)

In [45]:
feature_names = [col.replace(' ', '') for col in X_tr.columns]
feature_names

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [7]:
target_name = 'Diabetes'
target_labels = ['Good', 'Bad']

In [8]:
feature_constraints = ['' for i in range(len(feature_names))]
len(feature_constraints) == X_tr.shape[1]

True

In [9]:
feature_categories = []

In [10]:
feature_types = 5*['I'] + 2*['C'] + ['I']
feature_types

['I', 'I', 'I', 'I', 'I', 'C', 'C', 'I']

In [11]:
X_tr = X_tr.values
X_ts = X_ts.values

In [12]:
y_tr = y_tr.values
y_ts = y_ts.values

In [13]:
X = X_tr[mdl.predict(X_tr)==1]
X_vl = X_ts[mdl.predict(X_ts)==1]

## Actionable Recourse Summary

In [25]:
print('## Actionable Recourse Summary')
ares = AReS(mdl, X_tr, max_rule=6, max_rule_length=4, discretization_bins=20, minimum_support=0.05, print_objective=False,
            feature_names=feature_names, feature_types=feature_types, feature_categories=feature_categories, 
            feature_constraints=feature_constraints, target_name=target_name, target_labels=target_labels)
ares = ares.fit(X, max_change_num=4, cost_type='MPS', lambda_acc=1.0, lambda_cov=1.0, lambda_cst=1.0)
print('* Parameters:')
print('\t* lambda_acc: {}'.format(ares.lambda_acc)); print('\t* lambda_cov: {}'.format(ares.lambda_cov)); print('\t* lambda_cst: {}'.format(ares.lambda_cst));
print('\t* minimum support: {}'.format(ares.rule_miner_.minsup_)); print('\t* discretization bins: {}'.format(ares.rule_miner_.fd_.bins)); print('\t* pre-processing time[s]: {}'.format(ares.preprocess_time_)); 
print('\t* max rule: {}'.format(ares.max_rule_)); print('\t* max rule length: {}'.format(ares.max_rule_length_)); print('\t* Time[s]:', ares.time_); 
print('\t* uncover test: {}'.format(ares.uncover(X_vl))); print('\t* conflict: {}'.format(ares.conflict(X_vl))); print();
print('### Learned AReS')
print(ares.to_markdown())

## Actionable Recourse Summary
* Parameters:
	* lambda_acc: 1.0
	* lambda_cov: 1.0
	* lambda_cst: 1.0
	* minimum support: 0.05
	* discretization bins: 20
	* pre-processing time[s]: 0.0029807170012645656
	* max rule: 6
	* max rule length: 4
	* Time[s]: 3.8792489529987506
	* uncover test: 0.49056603773584906
	* conflict: 0.07407407407407407

### Learned AReS
| | Rule | Action |
| :---: | --- | --- |
| Recourse <br> rule 1 <br> (probability: 15.6%) | If '158.0<=x_1<172.0' | 89.0<=x_1<93.0 |
| Recourse <br> rule 2 <br> (probability: 7.5%) | If '136.0<=x_1<141.6' | 89.0<=x_1<93.0 |
| Recourse <br> rule 3 <br> (probability: 8.4%) | If '125.0<=x_1<130.0' | 100.0<=x_1<103.0 |
| Recourse <br> rule 4 <br> (probability: 9.0%) | If '34.7<=x_5<35.7' | 26.0<=x_5<27.3 |
| Recourse <br> rule 5 <br> (probability: 2.9%) | If '112.0<=x_1<117.0' | 89.0<=x_1<93.0 |
| Recourse <br> rule 6 <br> (probability: 5.8%) | If '39.4<=x_5<41.3' | 26.0<=x_5<27.3 |
| Default <br> rule | Else | 89.0<=x_1<93.0 |



In [47]:
x_ares = ares.predict(X_vl) + X_vl
y_ares = mdl.predict(x_ares)

print('Positive accuracy of AReS = {}'.format(np.mean(y_ares != mdl.predict(X_vl))))
print('Negative Inlier AReS = {}'.format(np.mean(isolation.predict(x_ares) == 1)))

Positive accuracy of AReS = 0.5660377358490566
Negative Inlier AReS = 0.7075471698113207


In [48]:
print('Sparsity', np.mean(np.sum(x_ares-X_vl!=0, axis=1)))

Sparsity 1.0


# Counterfactual TREE

In [29]:
print('## Counterfactual Explanation Tree')
LAMBDA = 0.01
GAMMA = 1.0

cet = CounterfactualExplanationTree(mdl, X_tr, y_tr, max_iteration=500, lime_approximation=(model!='L'),
                                    feature_names=feature_names, feature_types=feature_types, feature_categories=feature_categories,
                                    feature_constraints=feature_constraints, target_name=target_name, target_labels=target_labels)

cet = cet.fit(X, max_change_num=8, cost_type='MPS', C=LAMBDA, gamma=GAMMA, max_leaf_size=-1, time_limit=180, verbose=True)
print('* Parameters:'); print('\t* lambda: {}'.format(cet.lambda_)); print('\t* gamma: {}'.format(cet.gamma_)); print('\t* max_iteration: {}'.format(cet.max_iteration_));
print('\t* leaf size bound:', cet.leaf_size_bound_); print('\t* leaf size:', cet.n_leaves_); print('\t* LIME approximation:', cet.lime_approximation_); print('\t* Time[s]:', cet.time_); print();
print('### Learned CET')
cet.print_tree()

## Counterfactual Explanation Tree
## Stochastic Local Searching ...
### Iteration: 10
#### Before:
- If DiabetesPedigreeFunction<0.687:
	- If DiabetesPedigreeFunction<0.2242:
		* Action [Diabetes: Bad -> Good] (48/49 = 98.0% / MeanCost = 0.575):
			* Glucose: -62
			* BloodPressure: +15
			* Insulin: +58
			* BMI: -13.0160
			* DiabetesPedigreeFunction: -0.0100
			* Age: +6
	- Else:
		* Action [Diabetes: Bad -> Good] (53/178 = 29.8% / MeanCost = 0.115):
			* BloodPressure: +4
			* Insulin: +5
			* BMI: -2.4440
			* DiabetesPedigreeFunction: -0.0543
- Else:
	- If BMI<30.1:
		* Action [Diabetes: Bad -> Good] (20/20 = 100.0% / MeanCost = 0.335):
			* Pregnancies: -2
			* Glucose: -19
			* BloodPressure: +8
			* Insulin: +21
			* BMI: -22.6760
			* DiabetesPedigreeFunction: +1.1241
			* Age: +8
	- Else:
		- If Age<42:
			- If Pregnancies<2:
				* Action [Diabetes: Bad -> Good] (29/29 = 100.0% / MeanCost = 0.796):
					* Glucose: -110
					* BloodPressure: +24
					* Insulin: +104
					* B

### Iteration: 30
#### Before:
- If DiabetesPedigreeFunction<0.687:
	- If DiabetesPedigreeFunction<0.3042:
		* Action [Diabetes: Bad -> Good] (97/97 = 100.0% / MeanCost = 0.603):
			* Glucose: -65
			* BloodPressure: +8
			* Insulin: +50
			* BMI: -13.0160
			* DiabetesPedigreeFunction: -0.0100
			* Age: +14
	- Else:
		- If Pregnancies<4:
			* Action [Diabetes: Bad -> Good] (50/50 = 100.0% / MeanCost = 0.639):
				* Glucose: -74
				* BloodPressure: +12
				* Insulin: +94
				* BMI: -12.1160
				* DiabetesPedigreeFunction: -0.1892
				* Age: +9
		- Else:
			- If Insulin<155:
				- If Age<33:
					* Action [Diabetes: Bad -> Good] (12/12 = 100.0% / MeanCost = 0.505):
						* Pregnancies: -4
						* Glucose: -43
						* BloodPressure: +33
						* SkinThickness: +25
						* Insulin: +74
						* BMI: -10.1680
						* DiabetesPedigreeFunction: +0.5931
						* Age: -1
				- Else:
					- If Glucose<94:
						* Action [Diabetes: Bad -> Good] (3/3 = 100.0% / MeanCost = 0.11):
							* Pregnancie

### Iteration: 40
#### Before:
- If DiabetesPedigreeFunction<0.687:
	- If DiabetesPedigreeFunction<0.3042:
		* Action [Diabetes: Bad -> Good] (97/97 = 100.0% / MeanCost = 0.603):
			* Glucose: -65
			* BloodPressure: +8
			* Insulin: +50
			* BMI: -13.0160
			* DiabetesPedigreeFunction: -0.0100
			* Age: +14
	- Else:
		- If Pregnancies<4:
			* Action [Diabetes: Bad -> Good] (50/50 = 100.0% / MeanCost = 0.639):
				* Glucose: -74
				* BloodPressure: +12
				* Insulin: +94
				* BMI: -12.1160
				* DiabetesPedigreeFunction: -0.1892
				* Age: +9
		- Else:
			- If Insulin<155:
				- If Age<33:
					* Action [Diabetes: Bad -> Good] (12/12 = 100.0% / MeanCost = 0.505):
						* Pregnancies: -4
						* Glucose: -43
						* BloodPressure: +33
						* SkinThickness: +25
						* Insulin: +74
						* BMI: -10.1680
						* DiabetesPedigreeFunction: +0.5931
						* Age: -1
				- Else:
					- If Glucose<94:
						* Action [Diabetes: Bad -> Good] (3/3 = 100.0% / MeanCost = 0.11):
							* Pregnancie

### Iteration: 50
#### Before:
- If BMI<33.7:
	- If Age<33:
		* Action [Diabetes: Bad -> Good] (29/29 = 100.0% / MeanCost = 0.469):
			* Glucose: -23
			* BloodPressure: +16
			* Insulin: +37
			* BMI: -23.0640
			* DiabetesPedigreeFunction: -0.0630
			* Age: -1
	- Else:
		- If Pregnancies<2:
			* Action [Diabetes: Bad -> Good] (7/7 = 100.0% / MeanCost = 0.306):
				* Pregnancies: -1
				* Glucose: -27
				* BMI: -7.2800
				* DiabetesPedigreeFunction: +0.1719
				* Age: -5
		- Else:
			- If Insulin<155:
				- If SkinThickness<27:
					- If Pregnancies<6:
						* Action [Diabetes: Bad -> Good] (14/14 = 100.0% / MeanCost = 0.401):
							* Pregnancies: -2
							* Glucose: -46
							* BloodPressure: +9
							* Insulin: +38
							* BMI: -5.8320
							* DiabetesPedigreeFunction: -0.0570
							* Age: -8
					- Else:
						* Action [Diabetes: Bad -> Good] (26/29 = 89.7% / MeanCost = 0.377):
							* Pregnancies: -2
							* Glucose: -20
							* BloodPressure: +5
							* Insulin: +13


- If BMI<33.7:
	- If Age<33:
		* Action [Diabetes: Bad -> Good] (29/29 = 100.0% / MeanCost = 0.469):
			* Glucose: -23
			* BloodPressure: +16
			* Insulin: +37
			* BMI: -23.0640
			* DiabetesPedigreeFunction: -0.0630
			* Age: -1
	- Else:
		- If Pregnancies<2:
			* Action [Diabetes: Bad -> Good] (7/7 = 100.0% / MeanCost = 0.306):
				* Pregnancies: -1
				* Glucose: -27
				* BMI: -7.2800
				* DiabetesPedigreeFunction: +0.1719
				* Age: -5
		- Else:
			- If Insulin<155:
				- If SkinThickness<27:
					- If BloodPressure<68:
						* Action [Diabetes: Bad -> Good] (15/15 = 100.0% / MeanCost = 0.401):
							* Pregnancies: -3
							* Glucose: -33
							* BloodPressure: +12
							* SkinThickness: +17
							* Insulin: +31
							* BMI: -8.1800
							* DiabetesPedigreeFunction: -0.0570
							* Age: -10
					- Else:
						* Action [Diabetes: Bad -> Good] (25/28 = 89.3% / MeanCost = 0.399):
							* Pregnancies: -2
							* Glucose: -23
							* BloodPressure: +6
							* Insulin: +1

...

### Iteration: 460
#### Before:
- If BMI<37.9:
	- If Insulin<70:
		- If BMI<30.1:
			* Action [Diabetes: Bad -> Good] (25/30 = 83.3% / MeanCost = 0.314):
				* Pregnancies: -2
				* Glucose: -21
				* BloodPressure: +6
				* Insulin: +15
				* BMI: -24.3000
				* DiabetesPedigreeFunction: +0.0874
				* Age: +6
		- Else:
			- If BloodPressure<68:
				* Action [Diabetes: Bad -> Good] (22/22 = 100.0% / MeanCost = 0.584):
					* Glucose: -63
					* BloodPressure: +14
					* Insulin: +76
					* BMI: -10.7560
					* DiabetesPedigreeFunction: -0.0490
					* Age: +16
			- Else:
				* Action [Diabetes: Bad -> Good] (69/70 = 98.6% / MeanCost = 0.623):
					* Glucose: -38
					* BloodPressure: +18
					* Insulin: +56
					* BMI: -30.4000
					* DiabetesPedigreeFunction: -0.0100
					* Age: +5
	- Else:
		- If Glucose<94:
			* Action [Diabetes: Bad -> Good] (2/2 = 100.0% / MeanCost = 0.087):
				* Glucose: -80
				* BloodPressure: +2
				* SkinThickness: -3
				* Insulin: +21
				* BMI: -0.3480
			

### Iteration: 470
#### Before:
- If BMI<37.9:
	- If Insulin<70:
		- If BMI<30.1:
			* Action [Diabetes: Bad -> Good] (25/30 = 83.3% / MeanCost = 0.314):
				* Pregnancies: -2
				* Glucose: -21
				* BloodPressure: +6
				* Insulin: +15
				* BMI: -24.3000
				* DiabetesPedigreeFunction: +0.0874
				* Age: +6
		- Else:
			- If Pregnancies<2:
				* Action [Diabetes: Bad -> Good] (19/19 = 100.0% / MeanCost = 0.545):
					* Glucose: -56
					* BloodPressure: +18
					* Insulin: +70
					* BMI: -9.1440
					* DiabetesPedigreeFunction: -0.0100
					* Age: +10
			- Else:
				* Action [Diabetes: Bad -> Good] (73/73 = 100.0% / MeanCost = 0.478):
					* Pregnancies: -2
					* Glucose: -42
					* BloodPressure: +15
					* Insulin: +56
					* BMI: -9.4440
					* DiabetesPedigreeFunction: -0.0430
					* Age: +2
	- Else:
		- If Pregnancies<4:
			* Action [Diabetes: Bad -> Good] (46/46 = 100.0% / MeanCost = 0.46):
				* Glucose: -55
				* BloodPressure: +12
				* SkinThickness: -13
				* Insulin: +

### Iteration: 480
#### Before:
- If BMI<37.9:
	- If Insulin<70:
		- If BMI<30.1:
			- If Glucose<124:
				* Action [Diabetes: Bad -> Good] (13/13 = 100.0% / MeanCost = 0.249):
					* Pregnancies: -2
					* Glucose: -19
					* BloodPressure: +8
					* SkinThickness: +5
					* Insulin: +16
					* BMI: -4.8320
					* DiabetesPedigreeFunction: -0.1050
					* Age: +4
			- Else:
				* Action [Diabetes: Bad -> Good] (12/17 = 70.6% / MeanCost = 0.287):
					* Pregnancies: -2
					* Glucose: -23
					* BloodPressure: +5
					* Insulin: +15
					* BMI: -22.6880
					* DiabetesPedigreeFunction: +0.0405
					* Age: +9
		- Else:
			- If Pregnancies<2:
				* Action [Diabetes: Bad -> Good] (19/19 = 100.0% / MeanCost = 0.545):
					* Glucose: -56
					* BloodPressure: +18
					* Insulin: +70
					* BMI: -9.1440
					* DiabetesPedigreeFunction: -0.0100
					* Age: +10
			- Else:
				* Action [Diabetes: Bad -> Good] (73/73 = 100.0% / MeanCost = 0.478):
					* Pregnancies: -2
					* Glucose: -42
					* 

### Iteration: 490
#### Before:
- If BMI<37.9:
	- If Insulin<70:
		- If BMI<30.1:
			- If DiabetesPedigreeFunction<0.3042:
				* Action [Diabetes: Bad -> Good] (10/11 = 90.9% / MeanCost = 0.225):
					* Pregnancies: -2
					* Glucose: -21
					* BloodPressure: +5
					* Insulin: +16
					* BMI: -4.9560
					* DiabetesPedigreeFunction: -0.0532
					* Age: +2
			- Else:
				* Action [Diabetes: Bad -> Good] (16/19 = 84.2% / MeanCost = 0.321):
					* Pregnancies: -2
					* Glucose: -21
					* BloodPressure: +8
					* Insulin: +15
					* BMI: -22.6760
					* Age: +15
		- Else:
			- If Pregnancies<2:
				* Action [Diabetes: Bad -> Good] (19/19 = 100.0% / MeanCost = 0.545):
					* Glucose: -56
					* BloodPressure: +18
					* Insulin: +70
					* BMI: -9.1440
					* DiabetesPedigreeFunction: -0.0100
					* Age: +10
			- Else:
				* Action [Diabetes: Bad -> Good] (73/73 = 100.0% / MeanCost = 0.478):
					* Pregnancies: -2
					* Glucose: -42
					* BloodPressure: +15
					* Insulin: +56
					* 

### Iteration: 500
#### Before:
- If BMI<37.9:
	- If Insulin<70:
		- If BMI<30.1:
			- If DiabetesPedigreeFunction<0.3042:
				* Action [Diabetes: Bad -> Good] (10/11 = 90.9% / MeanCost = 0.225):
					* Pregnancies: -2
					* Glucose: -21
					* BloodPressure: +5
					* Insulin: +16
					* BMI: -4.9560
					* DiabetesPedigreeFunction: -0.0532
					* Age: +2
			- Else:
				- If BloodPressure<74:
					* Action [Diabetes: Bad -> Good] (7/7 = 100.0% / MeanCost = 0.245):
						* Pregnancies: -2
						* Glucose: -17
						* BloodPressure: +3
						* SkinThickness: +15
						* Insulin: +16
						* BMI: -4.8320
						* DiabetesPedigreeFunction: -0.1106
						* Age: -5
				- Else:
					* Action [Diabetes: Bad -> Good] (12/12 = 100.0% / MeanCost = 0.306):
						* Pregnancies: -2
						* Glucose: -22
						* Insulin: +15
						* BMI: -21.0640
						* DiabetesPedigreeFunction: +1.1524
						* Age: +15
		- Else:
			- If Pregnancies<2:
				* Action [Diabetes: Bad -> Good] (19/19 = 100.0% / MeanCost 

In [49]:
x_cet = cet.predict(X_vl) + X_vl
y_cet = mdl.predict(x_cet)

print('Positive accuracy of CET = {}'.format(np.mean(y_cet != mdl.predict(X_vl))))
print('Positive Inlier CET = {}'.format(np.mean(isolation.predict(x_cet) == 1)))

Positive accuracy of CET = 0.8490566037735849
Positive Inlier CET = 0.5943396226415094


In [50]:
print('Sparsity', np.mean(np.sum(x_cet-X_vl!=0, axis=1)))

Sparsity 6.311320754716981


In [51]:
cet.print_tree()

- If Insulin<155:
	- If Pregnancies<2:
		* Action [Diabetes: Bad -> Good] (44/48 = 91.7% / MeanCost = 0.459):
			* Glucose: -46
			* BloodPressure: +12
			* Insulin: +37
			* BMI: -10.5040
			* DiabetesPedigreeFunction: +0.0368
	- Else:
		- If SkinThickness<18:
			- If Age<23:
				* Action [Diabetes: Bad -> Good] (7/7 = 100.0% / MeanCost = 0.198):
					* Pregnancies: -1
					* Glucose: -16
					* BloodPressure: +3
					* SkinThickness: +5
					* Insulin: +16
					* BMI: -4.2200
					* DiabetesPedigreeFunction: -0.0873
					* Age: -1
			- Else:
				- If Pregnancies<6:
					* Action [Diabetes: Bad -> Good] (39/40 = 97.5% / MeanCost = 0.425):
						* Pregnancies: -2
						* Glucose: -36
						* BloodPressure: +9
						* SkinThickness: +7
						* Insulin: +26
						* BMI: -9.0560
						* DiabetesPedigreeFunction: +0.1384
						* Age: -2
				- Else:
					* Action [Diabetes: Bad -> Good] (46/46 = 100.0% / MeanCost = 0.557):
						* Pregnancies: -4
						* Glucose: -37
						* BloodPressure: 